In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb

!pip install swig
!pip install cvxpy
!pip install box2d-py
!pip uninstall pyglet -y
!pip uninstall gym -y
!pip install pyglet==1.3.2
!pip install gym==0.9.4
!pip install pyvirtualdisplay
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.15
!apt install --allow-change-held-packages libcudnn7=7.4.1.5-1+cuda10.0

In [ ]:
!git clone -b switch https://github.com/naufalhisyam/rocket-lander.git
%cd /content/rocket-lander
!ls

# Settings

In [ ]:
NAME = 'ferrante' #Saved model name
individu = 29 # see genesDict below for reference
test_num = 'tes_angin_konstan4' #

genesDict = {
  17: [0,0,0,0,1,0,0,0,1,1],
  29: [1,1,1,1,1,1,1,1,1,1],
  36: [2,1,1,1,1,1,1,1,1,1],
  52: [abs(1)/100,1,abs(1)/10,1,1,1,1,1,1,1],
}

genes = genesDict.get(individu)
if len(genes) != 10:
  raise Exception("incorrect genes size")

print(genes)

In [ ]:
#The height where the controller switch to PID
PID_SWITCH_HEIGHT = 5.72 

#Wind Properties
SIMULATE_WIND = True
total_windstep = 50 #gives wind for total step
wind_dir = 'right' #wind direction : 'left' or 'right'
x_force = 5 # x-axis wind force in Newton
y_force = 0 # y-axis wind force in Newton

# Functions

In [1]:
def ckpt_path(model_dir, NAME):
    with open(f"{model_dir}/checkpoint",'r+') as f:
        # Reading the file data and store
        # it in a file variable
        file = f.read()
            
        # Replacing the pattern with the string
        # in the file data
        file = re.sub(r'DDPG/(.*?)/DDPG', f"DDPG/{NAME}/DDPG", file)

        # Setting the position to the top
        # of the page to insert data
        f.seek(0)
            
        # Writing replaced data in the file
        f.write(file)

        # Truncating the file size
        f.truncate()

In [2]:
def plot(state_or_action,data):
    #TEST RESULT: STATE
    data_df = data
    data_df['step'] = data_df.index
    titles = {'step':'Waktu','x_pos':'Posisi Horizontal','y_pos':'Posisi Vertikal','x_vel':'Kepatan Horizontal','y_vel':'Kecepatan Vertikal',
                'angle_deg':'Sudut Roket','angvel_deg':'Kecepatan Sudut','remaining_fuel':'Bahan Bakar',
                'Fe':'Pendorong Bawah','Fs':'Pendorong Samping','Psi_deg':'Sudut Nozzle'}
    labels = {'step':'Waktu (s)','x_pos':'Posisi Horizontal (m)','y_pos':'Posisi Vertikal (m)','x_vel':'Kepatan Horizontal (m/s)','y_vel':'Kecepatan Vertikal (m/s)',
                'angle_deg':'Sudut Roket (deg)','angvel_deg':'Kecepatan Sudut (deg/s)','remaining_fuel':'Bahan Bakar (kg)',
                'Fe':'Pendorong Bawah (N)','Fs':'Pendorong Samping (N)','Psi_deg':'Sudut Nozzle (deg)'}
    
    def main_plot(xx,yy):
      images_dir = f"{model_dir}/{test_num}/"
      if not os.path.exists(images_dir):
        os.makedirs(images_dir)
      xdata = data_df[xx]
      ydata = data_df[yy]
      xlabel = labels.get(xx)
      ylabel = labels.get(yy)
      xtitle = titles.get(xx)
      ytitle = titles.get(yy)

      import matplotlib
      from matplotlib import pyplot as plt
      matplotlib.use('Agg')
      
      filename = f"{xx}_{yy}"
      title = f"Individu {individu}\n{xtitle} terhadap {ytitle}"
      plt.rcdefaults()
      plt.clf()
      plt.plot(xdata,ydata,'k')
      plt.xlabel(xlabel)
      plt.ylabel(ylabel)
      plt.title(title)
      plt.grid()
      plt.savefig(f"{images_dir}" + filename + ".svg")
      plt.savefig(f"{images_dir}" + filename + ".png")
      plt.show()

    if state_or_action == 'state': 
        main_plot('x_pos','y_pos')
        main_plot('step','x_pos')
        main_plot('step','y_pos')
        main_plot('step','x_vel')
        main_plot('step','y_vel')
        main_plot('step','angle_deg')
        main_plot('step','angvel_deg')
        main_plot('step','remaining_fuel')
    elif state_or_action == 'action':
        main_plot('step','Fe')
        main_plot('step','Fs')
        main_plot('step','Psi_deg')

In [3]:
def test(env, agent, pid, x_force, y_force):
    obs_size = env.observation_space.shape[0]

    util = Utils()
    state_samples = get_state_sample(samples=5000, genes=genes, normal_state=True)
    util.create_normalizer(state_sample=state_samples)
    xpos, ypos, xvel, yvel, lander_angle, angular_vel, rem_fuel, lander_mass, xpos_rocket, ypos_rocket, xpos_landingPad, ypos_landingPad = ([] for _ in range(12))
    fE, fS, pSi = ([] for _ in range(3))

    angle_deg, angvel_deg = [],[]
    psi_deg = []

    rew, total_rew = [], []
    shaping_pos, shaping_vel = [], []
    shaping_ang, shaping_angvel = [], []
    shaping_leftleg, shaping_rightleg = [], []
    shaping_fe, shaping_fs = [], []
    ep = []

    if SIMULATE_WIND:
        wind_x, wind_y = ([] for _ in range(2))
    
    for episode in range(1, NUM_EPISODES + 1):
        old_state = None
        done = False
        total_reward = 0

        state = env.reset()
        state = util.normalize(state)
        max_steps = 500

        left_or_right_barge_movement = np.random.randint(0, 2)
        
        pid_switch = False #flag to denormalize state when switching to PID

        for t in range(max_steps): # env.spec.max_episode_steps
            old_state = state
            
            current_state = env.get_state_with_barge_and_landing_coordinates(untransformed_state=True)
            current_y = current_state[1] - current_state[13] 
                
            xpos.append(current_state[0]-current_state[12]) #xpos_rocket - xpos_landingPad
            ypos.append(current_state[1]-current_state[13]) #ypos_rocket - ypos_landingPad
            xvel.append(current_state[2]) #xdot
            yvel.append(current_state[3]) #ydot
            lander_angle.append(current_state[4]) #theta
            angular_vel.append(current_state[5]) #theta_dot
            angle_deg.append(np.degrees(current_state[4]))
            angvel_deg.append(np.degrees(current_state[5]))
            rem_fuel.append(current_state[6]) #initial fuel = 0.2 * initial_mass
            lander_mass.append(current_state[7]) #initial_mass = 25.222
            xpos_rocket.append(current_state[0]) # xpos_rocket
            ypos_rocket.append(current_state[1]) # ypos_rocket
            xpos_landingPad.append(current_state[12]) # xpos_landingPad
            ypos_landingPad.append(current_state[13]) # ypos_landingPad
            fE.append(action[0][0])
            fS.append(action[0][1])
            pSi.append(action[0][2])
            psi_deg.append(np.degrees(action[0][2]))

            # Use DDPG agent when the rocket is above the PID_SWITCH_HEIGHT
            if current_y > PID_SWITCH_HEIGHT:
                # infer an action
                action = agent.get_action(np.reshape(state, (1, obs_size)), not TEST)
                # Pass the action to the env and step through the simulation (1 step)
                state, reward, done, _ = env.step(action[0])
                state = util.normalize(state)
                # print("Using DDPG")
        
            # Use PID when the rocket is at or below the PID_SWITCH_HEIGHT    
            else:
                # print("Using PID")
                if pid_switch == False:
                    # print("Denormalizing State")
                    state = util.denormalize(state)
                    pid_switch = True
                    
                # pass the state to the algorithm, get the actions
                action = pid.pid_algorithm(state) 
                # Pass the action to the env and step through the simulation (1 step).
                # Refer to Simulation Update in constants.py
                state, reward, done, _ = env.step(action)

            total_reward += reward
            
            rew.append(reward)
            shaping_pos.append(shaping_element[0])
            shaping_vel.append(shaping_element[1])
            shaping_ang.append(shaping_element[2])
            shaping_angvel.append(shaping_element[3])
            shaping_leftleg.append(shaping_element[4])
            shaping_rightleg.append(shaping_element[5])
            shaping_fe.append(shaping_element[6])
            shaping_fs.append(shaping_element[7])

            if SIMULATE_WIND:
                if state[LEFT_GROUND_CONTACT] == 0 and state[RIGHT_GROUND_CONTACT] == 0:
                    if t <= total_windstep:
                        env.apply_random_x_disturbance(epsilon=2, left_or_right=wind_dir, x_force=x_force)
                        if SAVE_TO_EXCEL:
                            winds = env.get_winds_value()
                            wind_x.append(winds[0])
                            wind_y.append(winds[1])
            

            if done:
                break

        # agent.log_data(total_reward, episode)

        print("Episode:\t{0}\tReward:\t{1}".format(episode, total_reward))
    
        if SAVE_REWARD:
            total_rew.append(total_reward)
            ep.append(episode)

    if SAVE_REWARD:
        os.makedirs(model_dir, exist_ok=True)
        tot_reward=pd.DataFrame(list(zip(ep,total_rew)),
                                 columns=['episode','total_reward'])
        with pd.ExcelWriter(model_dir + f"/DDPG_test-eps-rewards_{NAME}_{test_num}_{total_reward}_{len(ep)}.xlsx") as writer:
            tot_reward.to_excel(writer, sheet_name=f"{NAME}_test-eps-rewards")

        reward_data=pd.DataFrame(list(zip(rew,
                                          shaping_pos,
                                          shaping_vel,
                                          shaping_ang,
                                          shaping_angvel,
                                          shaping_leftleg,
                                          shaping_rightleg,
                                          shaping_fe,
                                          shaping_fs)),
                                 columns=['reward',
                                          'shaping_pos',
                                          'shaping_vel',
                                          'shaping_ang',
                                          'shaping_angvel',
                                          'shaping_leftleg',
                                          'shaping_rightleg',
                                          'r_fe',
                                          'r_fs'])
        with pd.ExcelWriter(model_dir + f"/DDPG_test-reward-element_{NAME}_{test_num}_{total_reward}_{len(ep)}.xlsx") as writer:
            reward_data.to_excel(writer, sheet_name=f"{NAME}_test-eps-rewards")

    if SAVE_TO_EXCEL:
        os.makedirs(model_dir, exist_ok=True)
        state_data=pd.DataFrame(list(zip(xpos,ypos,xvel,yvel,lander_angle,angular_vel,angle_deg,angvel_deg,rem_fuel,lander_mass,xpos_rocket,ypos_rocket,xpos_landingPad,ypos_landingPad)),\
            columns=['x_pos','y_pos','x_vel','y_vel','lateral_angle','angular_velocity','angle_deg','angvel_deg','remaining_fuel','lander_mass','xpos_rocket','ypos_rocket','xpos_landingPad','ypos_landingPad'])
        action_data=pd.DataFrame(list(zip(fE,fS,pSi,psi_deg)),columns=['Fe','Fs','Psi','Psi_deg'])
        if SIMULATE_WIND:
            wind_dat= pd.DataFrame(list(zip(wind_x, wind_y)),columns=['x_wind force', 'y_wind force'])
                
        with pd.ExcelWriter(model_dir + f"/DDPG_{NAME}_{test_num}_{total_reward}.xlsx") as writer:
            state_data.to_excel(writer, sheet_name="state")
            action_data.to_excel(writer, sheet_name="action")
            if SIMULATE_WIND:
                wind_dat.to_excel(writer, sheet_name="winds")    
        print("Steps:", len(state_data)-1)
        print("Logs saved")
    
    env.close()
    
    # fig, ax = plt.subplots()
    # ax.set_ylim(0,21)
    # ax.set_xlim(-10,10)
    # ax.plot(xpos, ypos, color='maroon')
    # ax.axvline(x=0, c='gray', linewidth=0.7)
    # ax.set_title('Rocket landing trajectory')
    # ax.set_xlabel('x position (m)')
    # ax.set_ylabel('y position (m)')
    # ax.grid()
    virtual_display.stop()
    print(f"X miss distance : {xpos[-1]}\nY miss distance : {ypos[-1]}")
    print(f"last X velocity : {xvel[-1]}\nlast Y velocity : {yvel[-1]}")
    print(f"last angle : {lander_angle[-1]}\nlast angular velocity : {angular_vel[-1]}")
    plot('state',state_data)
    plot('action',action_data)

# Imports

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1000, 800))
virtual_display.start()


In [ ]:
%tensorflow_version 1.x
%cd /content/rocket-lander

import os
import numpy as np
from numpy.core.numeric import False_
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
%matplotlib inline

from control_and_ai.DDPG.ddpg import DDPG
from control_and_ai.DDPG.utils import Utils
from control_and_ai.DDPG.exploration import OUPolicy

from control_and_ai.pid import PID_Benchmark

from constants import *
from constants import DEGTORAD
from environments.rocketlander_test_ga_constwind import RocketLander, get_state_sample

action_bounds = [1, 1, 15*DEGTORAD]

eps = []
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))

simulation_settings = {'Side Engines': True,
                       'Clouds': True,
                       'Vectorized Nozzle': True,
                       'Graph': False,
                       'Render': False,
                       'Starting Y-Pos Constant': 1,
                       'Initial Force': 'random',
                       'Rows': 1,
                       'Columns': 2,
                       'Episodes': 500,
                       'Genes': genes}

env = RocketLander(simulation_settings)
#env = wrappers.Monitor(env, './rocket_videos', force=True)

#Set both line below to False if you want to contniue training from a saved checkpoint
RETRAIN = False#Restore weights if False
TEST = True #Test the model

NUM_EPISODES = 1
SAVE_TO_EXCEL = True #Export states & actions logs as .xlsx
SAVE_REWARD = True

model_dir = '/content/gdrive/MyDrive/colab_model/rocket/DDPG/' + NAME
ckpt_path(model_dir,NAME)

agent = DDPG(
    action_bounds,
    eps,
    env.observation_space.shape[0], #for first model
    actor_learning_rate=0.001,
    critic_learning_rate=0.01,
    retrain=RETRAIN,
    log_dir="./logs",
    model_dir=model_dir,
    batch_size=100,
    gamma=0.99)

#Initialize PID algorithm
pid = PID_Benchmark()

# Main

In [ ]:
print('NAME:',NAME)
print('test_num:',test_num)
test(env, agent, pid, x_force, y_force)